In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Getting the dynamic table
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

url = 'https://csrankings.org/#/fromyear/2012/toyear/2022/index?all&us'
driver.get(url)

driver.implicitly_wait(10)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

html = soup.prettify()

with open('main.html', 'w') as f:
    f.write(html)
    

In [ ]:
# Getting University Information
from bs4 import BeautifulSoup
import re

with open('main.html', 'r') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table', id='ranking')

with open('table.html', 'w') as f:
    f.write(str(table))
    

with open('table.html', 'r') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table', id='ranking')

tbody = table.find('tbody')
rows = tbody.select('tr:not(tr tr)')

uni_infos = []
for i in range(0, len(rows), 3):

    row = rows[i]

    cells = row.find_all('td')
    rank = cells[0].get_text()
    name = cells[1].get_text()
    score = cells[2].get_text()
    faculty = cells[3].get_text()
    rank = ' '.join(rank.split())
    rank = int(re.sub(r'[^A-Za-z0-9 ]+', '', rank))
    name = ' '.join(name.split())
    name = re.sub(r'[^A-Za-z0-9 ]+', '', name).strip()
    score = ' '.join(score.split())
    score = float(re.sub(r'[^A-Za-z0-9. ]+', '', score))
    faculty = ' '.join(faculty.split())
    faculty = int(re.sub(r'[^A-Za-z0-9 ]+', '', faculty))
    info = [rank, name, score, faculty]
    uni_infos.append(info)

import pandas as pd
df = pd.DataFrame(uni_infos, columns =['rank', 'uni_name', 'count', 'faculty']) 


div_ids = [e.get('id') for e in soup.find_all(lambda tag: tag.get('id') and '-faculty' in tag.get('id'))]

clean_div_ids = [re.sub(r'[^A-Za-z]+', ' ', name).replace('faculty', '').strip() for name in div_ids]
import difflib

def find_best_match(name):
    return difflib.get_close_matches(name,div_ids, cutoff=0.3)[0]
df['div_ids'] = df['uni_name'].apply(find_best_match)

df.to_csv('uni_infos.csv')

data = []
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    print(len(cols))


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [ ]:

HTMLFile = open("main.html", "r", encoding="utf8")

index = HTMLFile.read()

S = BeautifulSoup(index, 'html.parser')

In [ ]:
univ_data = pd.read_csv("uni_infos.csv")
univ_data.drop("Unnamed: 0", axis=1, inplace=True)
univ_data.sample(10)

,rank,uni_name,count,faculty,div_ids
71,69,Worcester Polytechnic Institute,1.9,25,Worcester%20Polytechnic%20Institute-faculty
116,113,Texas State University,1.3,10,Texas%20State%20University-faculty
7,8,Cornell University,10.6,82,Cornell%20University-faculty
148,144,OHSU,1.1,3,OHSU-faculty
47,48,North Carolina State University,3.3,43,North%20Carolina%20State%20University-faculty
144,144,Cleveland State University,1.1,4,Cleveland%20State%20University-faculty
42,43,Rice University,3.5,30,Rice%20University-faculty
120,113,University of Kansas,1.3,12,University%20of%20Kansas-faculty
26,26,University of Southern California,5.1,47,University%20of%20Southern%20California-faculty
127,113,University of Texas at San Antonio,1.3,9,University%20of%20Texas%20at%20San%20Antonio-f...


In [ ]:
df = df[df['providing_assitantship']==1]
df = df[df['university_name']=='Florida State University']
df = df[df['faculty_areaname'] == 'hci']
print(df.loc[:, ['faculty_name', 'faculty_home_page', 'faculty_areaname', 'providing_assitantship']])

    faculty_name               faculty_home_page faculty_areaname  \
347     Jie Yang  http://www.cs.fsu.edu/~jieyang              hci   
350   Guang Wang            https://guangwang.me              hci   

     providing_assitantship  
347                       1  
350                       1  


In [ ]:
div_id = 'Carnegie%20Mellon%20University-faculty'
university = 'Carnegie Mellon University'

# DataFrame1 for faculty name, area, publications and university
# Getting the division of each university which is containing the faculty table
table = S.find('div', {'id': div_id})
df1 = pd.read_html(str(table))[0]
df1.drop('Unnamed: 0', axis=1, inplace=True)
df1 = df1.replace(to_replace='None', value=np.nan).dropna()
df1.columns = ['faculty_name_and_area', 'publications', 'adjusted']
df1['university_name'] = 'Carnegie Mellon University'
df1 = df1.reset_index()
df1.drop('index', axis=1, inplace=True)
df1



,faculty_name_and_area,publications,adjusted,university_name
0,Graham Neubig nlp,135.0,35.1,Carnegie Mellon University
1,Abhinav Gupta 0001 vision,111.0,32.3,Carnegie Mellon University
2,"David P. Woodruff ml , theory",99.0,33.7,Carnegie Mellon University
3,Howie Choset robotics,83.0,19.5,Carnegie Mellon University
4,"Tuomas Sandholm ai , ml",80.0,29.8,Carnegie Mellon University
...,...,...,...,...
161,Bruce H. Krogh,1.0,0.2,Carnegie Mellon University
162,Jack Mostow,1.0,0.5,Carnegie Mellon University
163,Priya Narasimhan,1.0,0.2,Carnegie Mellon University
164,Dana S. Scott,1.0,0.2,Carnegie Mellon University


In [ ]:
# DataFrame2 for faculty_homepage, dblp link
headers = []
data = []
for row in table.find_all('tr'):
    row_data = []
    for cell in row.find_all(['td', 'th']):
        link = cell.find('a')
        if link is not None:
            row_data.append(link.get('href'))
        else:
            row_data.append(cell.text.strip())
        if cell.name == 'a':
            if link is not None:
                headers.append('link_row'+ row_data[-1])
    if row_data:
        data.append(row_data)

df2 = pd.DataFrame(data)
df2.columns = df2.iloc[0]
df2 = df2[1:]
df2 = df2.replace(to_replace='None', value=np.nan).dropna()
df2 = df2.iloc[: , :-1]
df2 = df2.iloc[: , 1:]
df2.columns.values[1] = 'dblp'
df2 = df2.reset_index()
df2.drop('index', axis=1, inplace=True)
print(df2.columns)
df2.columns = ['faculty_homepage', 'dblp']
df2

Index(['Faculty', 'dblp'], dtype='object', name=0)


,faculty_homepage,dblp
0,http://www.phontron.com,https://dblp.org/pers/hd/n/Neubig:Graham
1,http://www.cs.cmu.edu/~abhinavg,https://dblp.org/pers/hd/g/Gupta_0001:Abhinav
2,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D
3,http://www.cs.cmu.edu/~./choset,https://dblp.org/pers/hd/c/Choset:Howie
4,http://www.cs.cmu.edu/~sandholm,https://dblp.org/pers/hd/s/Sandholm:Tuomas
...,...,...
161,https://www.ece.cmu.edu/directory/details/89,https://dblp.org/pers/hd/k/Krogh:Bruce_H%3D
162,http://www.cs.cmu.edu/~mostow,https://dblp.org/pers/hd/m/Mostow:Jack
163,http://www.cs.cmu.edu/~priya,https://dblp.org/pers/hd/n/Narasimhan:Priya
164,http://www.cs.cmu.edu/~scott,https://dblp.org/pers/hd/s/Scott:Dana_S%3D


In [ ]:
df = pd.concat([df1, df2], axis=1, join='inner')
print(df.columns)
df.columns = ["faculty_name_and_area", "publications", "adjusted", "university_name", "faculty_home_page", "dblp"]
df

Index(['faculty_name_and_area', 'publications', 'adjusted', 'university_name',
       'faculty_homepage', 'dblp'],
      dtype='object')


,faculty_name_and_area,publications,adjusted,university_name,faculty_home_page,dblp
0,Graham Neubig nlp,135.0,35.1,Carnegie Mellon University,http://www.phontron.com,https://dblp.org/pers/hd/n/Neubig:Graham
1,Abhinav Gupta 0001 vision,111.0,32.3,Carnegie Mellon University,http://www.cs.cmu.edu/~abhinavg,https://dblp.org/pers/hd/g/Gupta_0001:Abhinav
2,"David P. Woodruff ml , theory",99.0,33.7,Carnegie Mellon University,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D
3,Howie Choset robotics,83.0,19.5,Carnegie Mellon University,http://www.cs.cmu.edu/~./choset,https://dblp.org/pers/hd/c/Choset:Howie
4,"Tuomas Sandholm ai , ml",80.0,29.8,Carnegie Mellon University,http://www.cs.cmu.edu/~sandholm,https://dblp.org/pers/hd/s/Sandholm:Tuomas
...,...,...,...,...,...,...
161,Bruce H. Krogh,1.0,0.2,Carnegie Mellon University,https://www.ece.cmu.edu/directory/details/89,https://dblp.org/pers/hd/k/Krogh:Bruce_H%3D
162,Jack Mostow,1.0,0.5,Carnegie Mellon University,http://www.cs.cmu.edu/~mostow,https://dblp.org/pers/hd/m/Mostow:Jack
163,Priya Narasimhan,1.0,0.2,Carnegie Mellon University,http://www.cs.cmu.edu/~priya,https://dblp.org/pers/hd/n/Narasimhan:Priya
164,Dana S. Scott,1.0,0.2,Carnegie Mellon University,http://www.cs.cmu.edu/~scott,https://dblp.org/pers/hd/s/Scott:Dana_S%3D


In [ ]:
import pandas as pd
import re

def get_areas(s):
    lowercase_words = re.findall(r'\b[a-z]\w*\b', s)
    if len(lowercase_words) == 0:
        return  'unknown'
    else:
        return ','.join(lowercase_words)
def get_name(s):
    lowercase_words = re.findall(r'\b[a-z]\w*\b', s)
    if len(lowercase_words) == 0:
        return  s
    else:
        for word in lowercase_words:
            s = re.sub(r'\b{}\b'.format(word), '', s)
        return re.sub(r'[^a-zA-Z ]+', '', s).strip()
df['faculty_areaname'] = df['faculty_name_and_area'].apply(get_areas)
df['faculty_name_and_area'] = df['faculty_name_and_area'].apply(get_name)
df = df.rename(columns={"faculty_name_and_area": "faculty_name"})
df

,faculty_name,publications,adjusted,university_name,faculty_home_page,dblp,faculty_areaname
0,Graham Neubig,135.0,35.1,Carnegie Mellon University,http://www.phontron.com,https://dblp.org/pers/hd/n/Neubig:Graham,nlp
1,Abhinav Gupta,111.0,32.3,Carnegie Mellon University,http://www.cs.cmu.edu/~abhinavg,https://dblp.org/pers/hd/g/Gupta_0001:Abhinav,vision
2,David P Woodruff,99.0,33.7,Carnegie Mellon University,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D,"ml,theory"
3,Howie Choset,83.0,19.5,Carnegie Mellon University,http://www.cs.cmu.edu/~./choset,https://dblp.org/pers/hd/c/Choset:Howie,robotics
4,Tuomas Sandholm,80.0,29.8,Carnegie Mellon University,http://www.cs.cmu.edu/~sandholm,https://dblp.org/pers/hd/s/Sandholm:Tuomas,"ai,ml"
...,...,...,...,...,...,...,...
161,Bruce H. Krogh,1.0,0.2,Carnegie Mellon University,https://www.ece.cmu.edu/directory/details/89,https://dblp.org/pers/hd/k/Krogh:Bruce_H%3D,unknown
162,Jack Mostow,1.0,0.5,Carnegie Mellon University,http://www.cs.cmu.edu/~mostow,https://dblp.org/pers/hd/m/Mostow:Jack,unknown
163,Priya Narasimhan,1.0,0.2,Carnegie Mellon University,http://www.cs.cmu.edu/~priya,https://dblp.org/pers/hd/n/Narasimhan:Priya,unknown
164,Dana S. Scott,1.0,0.2,Carnegie Mellon University,http://www.cs.cmu.edu/~scott,https://dblp.org/pers/hd/s/Scott:Dana_S%3D,unknown


In [ ]:
print(df.columns)

Index(['faculty_name', 'publications', 'adjusted', 'university_name',
       'faculty_home_page', 'dblp', 'faculty_areaname'],
      dtype='object')


In [ ]:
new_df = df
new_df['faculty_areaname'] = new_df.faculty_areaname.str.split(',')
new_df = new_df.explode('faculty_areaname')

In [ ]:
new_df.head(10)

,faculty_name,publications,adjusted,university_name,faculty_home_page,dblp,faculty_areaname
0,Graham Neubig,135.0,35.1,Carnegie Mellon University,http://www.phontron.com,https://dblp.org/pers/hd/n/Neubig:Graham,nlp
1,Abhinav Gupta,111.0,32.3,Carnegie Mellon University,http://www.cs.cmu.edu/~abhinavg,https://dblp.org/pers/hd/g/Gupta_0001:Abhinav,vision
2,David P Woodruff,99.0,33.7,Carnegie Mellon University,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D,ml
2,David P Woodruff,99.0,33.7,Carnegie Mellon University,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D,theory
3,Howie Choset,83.0,19.5,Carnegie Mellon University,http://www.cs.cmu.edu/~./choset,https://dblp.org/pers/hd/c/Choset:Howie,robotics
4,Tuomas Sandholm,80.0,29.8,Carnegie Mellon University,http://www.cs.cmu.edu/~sandholm,https://dblp.org/pers/hd/s/Sandholm:Tuomas,ai
4,Tuomas Sandholm,80.0,29.8,Carnegie Mellon University,http://www.cs.cmu.edu/~sandholm,https://dblp.org/pers/hd/s/Sandholm:Tuomas,ml
5,Pradeep Ravikumar,79.0,20.8,Carnegie Mellon University,http://www.cs.cmu.edu/~pradeepr,https://dblp.org/pers/hd/r/Ravikumar:Pradeep,ml
6,Martial Hebert,78.0,22.0,Carnegie Mellon University,http://www.cs.cmu.edu/~./hebert,https://dblp.org/pers/hd/h/Hebert:Martial,vision
7,Deva Ramanan,77.0,24.4,Carnegie Mellon University,http://www.cs.cmu.edu/~deva,https://dblp.org/pers/hd/r/Ramanan:Deva,vision


In [ ]:
print(len(new_df[new_df['faculty_areaname']=='unknown']))

13


In [ ]:
univ_data.columns

Index(['rank', 'uni_name', 'count', 'faculty', 'div_ids'], dtype='object')

In [ ]:

from tqdm import tqdm
all_df = []
for i in tqdm(range(len(univ_data))):
    
    div_id = univ_data.loc[i, 'div_ids']
    university = univ_data.loc[i, 'uni_name']
    print(university)
    # DataFrame1 for faculty name, area, publications and university
    # Getting the division of each university which is containing the faculty table
    table = S.find('div', {'id': div_id})
    df1 = pd.read_html(str(table))[0]
    df1.drop('Unnamed: 0', axis=1, inplace=True)
    df1 = df1.replace(to_replace='None', value=np.nan).dropna()
    df1.columns = ['faculty_name_and_area', 'publications', 'adjusted']
    df1['university_name'] = university
    df1 = df1.reset_index()
    df1.drop('index', axis=1, inplace=True)
    # DataFrame2 for faculty_homepage, dblp link
    headers = []
    data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['td', 'th']):
            # Extract hyperlink if it exists
            link = cell.find('a')
            if link is not None:
                row_data.append(link.get('href'))
            else:
                row_data.append(cell.text.strip())

            # Extract header if it exists
            if cell.name == 'a':
                if link is not None:
                    headers.append('link_row'+ row_data[-1])
        if row_data:
            data.append(row_data)
    df2 = pd.DataFrame(data)
    df2.columns = df2.iloc[0]
    df2 = df2[1:]
    df2 = df2.replace(to_replace='None', value=np.nan).dropna()
    df2 = df2.iloc[: , :-1]
    df2 = df2.iloc[: , 1:]
    df2.columns.values[1] = 'dblp'
    df2 = df2.reset_index()
    df2.drop('index', axis=1, inplace=True)
    df2.columns = ['faculty_homepage', 'dblp']
    df = pd.concat([df1, df2], axis=1, join='inner')
    df.columns = ["faculty_name_and_area", "publications", "adjusted", "university_name", "faculty_home_page", "dblp"]
    df['faculty_areaname'] = df['faculty_name_and_area'].apply(get_areas)
    df['faculty_name_and_area'] = df['faculty_name_and_area'].apply(get_name)
    df = df.rename(columns={"faculty_name_and_area": "faculty_name"})
    new_df = df
    new_df['faculty_areaname'] = new_df.faculty_areaname.str.split(',')
    new_df = new_df.explode('faculty_areaname')
    all_df.append(new_df)

  0%|          | 0/187 [00:00<?, ?it/s]

Carnegie Mellon University


  1%|          | 1/187 [00:00<00:50,  3.70it/s]

Univ of Illinois at UrbanaChampaign


  1%|          | 2/187 [00:00<00:47,  3.93it/s]

...

 99%|█████████▉| 185/187 [02:43<00:02,  1.07s/it]

University of Oklahoma


 99%|█████████▉| 186/187 [02:45<00:01,  1.06s/it]

University of Tulsa


100%|██████████| 187/187 [02:46<00:00,  1.13it/s]


In [ ]:
all_df = pd.concat(all_df)

In [ ]:
all_df

,faculty_name,publications,adjusted,university_name,faculty_home_page,dblp,faculty_areaname
0,Graham Neubig,135.0,35.1,Carnegie Mellon University,http://www.phontron.com,https://dblp.org/pers/hd/n/Neubig:Graham,nlp
1,Abhinav Gupta,111.0,32.3,Carnegie Mellon University,http://www.cs.cmu.edu/~abhinavg,https://dblp.org/pers/hd/g/Gupta_0001:Abhinav,vision
2,David P Woodruff,99.0,33.7,Carnegie Mellon University,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D,ml
2,David P Woodruff,99.0,33.7,Carnegie Mellon University,http://www.cs.cmu.edu/~dwoodruf,https://dblp.org/pers/hd/w/Woodruff:David_P%3D,theory
3,Howie Choset,83.0,19.5,Carnegie Mellon University,http://www.cs.cmu.edu/~./choset,https://dblp.org/pers/hd/c/Choset:Howie,robotics
...,...,...,...,...,...,...,...
1,Christan Grant,1.0,0.2,University of Oklahoma,http://www.ou.edu/coe/cs/people/cgrant,https://dblp.org/pers/hd/g/Grant:Christan,unknown
2,Amy McGovern,1.0,0.2,University of Oklahoma,http://www.ou.edu/coe/cs/people/mcgovern,https://dblp.org/pers/hd/m/McGovern:Amy,unknown
3,Chris E. Weaver,1.0,0.5,University of Oklahoma,http://cs.ou.edu/~weaver/academic,https://dblp.org/pers/hd/w/Weaver:Chris_E%3D,unknown
0,Tyler Moore,4.0,1.1,University of Tulsa,https://tylermoore.ens.utulsa.edu,https://dblp.org/pers/hd/m/Moore_0001:Tyler,security


In [ ]:
print(len(all_df[all_df['faculty_areaname']=='hci']))

422


In [ ]:
all_df.to_csv('all_faculty_info.csv')

In [ ]:
div_ids = ['Carnegie%20Mellon%20University-faculty', 'Univ.%20of%20Illinois%20at%20Urbana-Champaign-faculty', 'Florida%20State%20University-faculty']
universities = ['Carnegie Mellon University', 'Univ of Illinois at UrbanaChampaign','Florida State University']

from tqdm import tqdm
three_df = []
for i in tqdm(range(3)):
    
    div_id = div_ids[i]
    university = universities[i]
    print(university)
    # DataFrame1 for faculty name, area, publications and university
    # Getting the division of each university which is containing the faculty table
    table = S.find('div', {'id': div_id})
    df1 = pd.read_html(str(table))[0]
    df1.drop('Unnamed: 0', axis=1, inplace=True)
    df1 = df1.replace(to_replace='None', value=np.nan).dropna()
    df1.columns = ['faculty_name_and_area', 'publications', 'adjusted']
    df1['university_name'] = university
    df1 = df1.reset_index()
    df1.drop('index', axis=1, inplace=True)
    # DataFrame2 for faculty_homepage, dblp link
    headers = []
    data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['td', 'th']):
            # Extract hyperlink if it exists
            link = cell.find('a')
            if link is not None:
                row_data.append(link.get('href'))
            else:
                row_data.append(cell.text.strip())

            # Extract header if it exists
            if cell.name == 'a':
                if link is not None:
                    headers.append('link_row'+ row_data[-1])
        if row_data:
            data.append(row_data)
    df2 = pd.DataFrame(data)
    df2.columns = df2.iloc[0]
    df2 = df2[1:]
    df2 = df2.replace(to_replace='None', value=np.nan).dropna()
    df2 = df2.iloc[: , :-1]
    df2 = df2.iloc[: , 1:]
    df2.columns.values[1] = 'dblp'
    df2 = df2.reset_index()
    df2.drop('index', axis=1, inplace=True)
    df2.columns = ['faculty_homepage', 'dblp']
    df = pd.concat([df1, df2], axis=1, join='inner')
    df.columns = ["faculty_name_and_area", "publications", "adjusted", "university_name", "faculty_home_page", "dblp"]
    df['faculty_areaname'] = df['faculty_name_and_area'].apply(get_areas)
    df['faculty_name_and_area'] = df['faculty_name_and_area'].apply(get_name)
    df = df.rename(columns={"faculty_name_and_area": "faculty_name"})
    new_df = df
    new_df['faculty_areaname'] = new_df.faculty_areaname.str.split(',')
    new_df = new_df.explode('faculty_areaname')
    three_df.append(new_df)
three_df = pd.concat(three_df)

  0%|          | 0/3 [00:00<?, ?it/s]

Carnegie Mellon University


 33%|███▎      | 1/3 [00:00<00:00,  3.72it/s]

Univ of Illinois at UrbanaChampaign


 67%|██████▋   | 2/3 [00:00<00:00,  3.98it/s]

Florida State University


100%|██████████| 3/3 [00:01<00:00,  1.79it/s]


In [ ]:
three_df.to_csv('three_uni.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('funding_info2.csv')
df.sample(10)

,Unnamed: 0,Unnamed: 0.1,faculty_name,publications,adjusted,university_name,faculty_home_page,dblp,faculty_areaname,providing_assitantship,matched_ratio,match_words
183,183,151,Seth Copen Goldstein,2.0,0.5,Carnegie Mellon University,http://www.cs.cmu.edu/~seth,https://dblp.org/pers/hd/g/Goldstein:Seth_Copen,robotics,0,0.000000,NaN
104,104,88,Virginia Smith,15.0,3.4,Carnegie Mellon University,http://users.ece.cmu.edu/~smithv,https://dblp.org/pers/hd/s/Smith:Virginia,ml,0,0.000000,NaN
115,115,97,Nihar B Shah,13.0,4.3,Carnegie Mellon University,http://www.cs.cmu.edu/~nihars,https://dblp.org/pers/hd/s/Shah:Nihar_B%3D,ai,0,0.000000,NaN
268,268,52,Matus Telgarsky,16.0,7.1,Univ of Illinois at UrbanaChampaign,http://mjt.cs.illinois.edu,https://dblp.org/pers/hd/t/Telgarsky:Matus,ml,0,0.000000,NaN
72,72,60,Norman M Sadeh,22.0,4.2,Carnegie Mellon University,http://www.normsadeh.org,https://dblp.org/pers/hd/s/Sadeh:Norman_M%3D,nlp,0,0.000000,NaN
336,336,105,Luke N. Olson,2.0,0.5,Univ of Illinois at UrbanaChampaign,http://lukeo.cs.illinois.edu,https://dblp.org/pers/hd/o/Olson:Luke_N%3D,unknown,0,0.000000,NaN
300,300,78,Lav R Varshney,8.0,1.9,Univ of Illinois at UrbanaChampaign,http://varshney.web.engr.illinois.edu,https://dblp.org/pers/hd/v/Varshney:Lav_R%3D,ai,0,0.000000,NaN
112,112,95,Andr Platzer,13.0,6.0,Carnegie Mellon University,http://symbolaris.com,https://dblp.org/pers/hd/p/Platzer:Andr%3Deacu...,logic,0,0.000000,NaN
79,79,66,Anupam Datta,20.0,4.9,Carnegie Mellon University,http://www.andrew.cmu.edu/user/danupam,https://dblp.org/pers/hd/d/Datta:Anupam,security,0,0.000000,NaN
49,49,44,Pravesh Kothari,27.0,8.1,Carnegie Mellon University,https://www.cs.cmu.edu/~praveshk,https://dblp.org/pers/hd/k/Kothari:Pravesh,theory,1,0.090909,"fellowships,"


In [1]:
# Querying Homepages
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
from urllib.error import URLError
import time
from selenium.common.exceptions import WebDriverException

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(3)

search_terms = ['openings', 'self-motivated', 'graduate student position', 'research assistantship', 'postdoctoral fellowship',
                'research grants', 'fellowships', 'scholarships', 'strong research skills', 'ability to work independently', 'prior research experience']

df = pd.read_csv('three_uni.csv')
df['providing_assitantship'] = 0
df['matched_ratio'] = 0
df['match_words'] = ""

for i in tqdm(range(len(df))):
    if i > 0 and df.loc[i-1, 'faculty_home_page'] == df.loc[i, 'faculty_home_page']:
        df.loc[i, 'providing_assitantship'] = df.loc[i-1, 'providing_assitantship']
        df.loc[i, 'matched_ratio'] = df.loc[i-1, 'matched_ratio']
    df.loc[i, 'faculty_home_page']
    url = df.loc[i, 'faculty_home_page']
    retry_count = 0
    while retry_count < 3:
        try:
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            html = soup.prettify()
            html = str(html).lower()
            count = 0
            matched = ""
            for s in search_terms:
                if s in html:
                    df.loc[i, 'providing_assitantship'] = 1
                    count += 1
                    matched = s + ', '
            df.loc[i, 'matched_ratio'] = count / len(search_terms)
            df.loc[i, 'match_words'] = matched
            break
        except (URLError, WebDriverException) as e:
            retry_count += 1
            print(f"Error: {e}. Retrying in 2 seconds...")
            time.sleep(2)
    if retry_count >= 3:
        print(f"Failed to retrieve data for URL: {url}")

driver.quit()


openings
self-motivated
graduate student position
research assistantship
postdoctoral fellowship
research grants
fellowships
scholarships
strong research skills
ability to work independently
prior research experience


In [ ]:
#Plotting
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('funding_info2.csv')
df = df[df['providing_assitantship']==1]

ax = df['faculty_areaname'].value_counts().plot.bar()
ax.bar_label(ax.containers[0], label_type="edge")
plt.title('Funding offer count in differecnt areas of three Universities: CMU, UIUC and FSU')
plt.xlabel('Available Offers')
plt.ylabel('Area')
plt.legend(loc="upper right")
plt.show()

all_df = pd.read_csv('all_faculty_info.csv')
all_df = all_df.drop(all_df[all_df['faculty_areaname'] == 'unknown'].index)

top_areas = all_df['faculty_areaname'].value_counts().head(10)
ax = top_areas.plot(kind='pie', autopct='%1.1f%%')
ax.set_ylabel('')
ax.legend().remove()
plt.title('Top 10 Areas with the Highest Faculty Population')
plt.show()

ml_faculties = all_df[all_df['faculty_areaname']=='ml']
ml_faculties = ml_faculties.sort_values('publications', ascending=False)
ml_faculties = ml_faculties.loc[:, ['faculty_name', 'publications', 'university_name']]
print(ml_faculties.head(7))

ml_faculties = ml_faculties.sort_values('publications', ascending=False)

top_faculties = ml_faculties.head(5)

ax = top_faculties.plot(kind='barh', x='faculty_name', y='publications')

for index, row in top_faculties.iterrows():
    ax.text(row['publications']+1, index, row['publications'], ha='left', va='center')
    ax.text(row['publications']+1, index-0.2, f"{row['publications']} ({index+1})", ha='left', va='center')


ax.set_xlabel('Number of Publications')

y_label = 'Faculty Name\nUniversity Name'
ax.set_ylabel(y_label)


for i, name in enumerate(top_faculties['university_name']):
    ax.text(0, i, name, ha='left', va='center')

ax.invert_yaxis()
ax.bar_label(ax.containers[0], label_type="edge")
plt.title('Top 5 Faculties in ML by Number of Publications (2012-2022)')

plt.show()
